In [82]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [83]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [84]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import MetaData
from sqlalchemy import create_engine, func

In [85]:
# create engine to hawaii.sqlite
engine = create_engine(f"sqlite:////Users/kellenquinn/Desktop/SQLALCHEMY CHALLENGE/Resources/hawaii.sqlite")
conn = engine.connect()

In [86]:
Base=automap_base()
Base.prepare(engine, reflect=True)

In [87]:
# reflect an existing database into a new model
from sqlalchemy import MetaData

#Get MetaData in Sqlite
m=MetaData()
m.reflect(engine)
# reflect the tables
for table in m.tables.values():
    print(table.name)

    for column in table.c:
        print(column.name)

measurement
id
station
date
prcp
tobs
station
id
station
name
latitude
longitude
elevation


In [88]:
# View all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [89]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [90]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [91]:
# Find the most recent date in the data set.
from sqlalchemy import desc
session.query(Measurement.date).order_by(Measurement.date.desc()).first()

('2017-08-23')

In [92]:
import datetime as dt

In [107]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 
session.query(Measurement.prcp).order_by(Measurement.prcp.desc()).first()
# Calculate the date one year from the last date in data set.
last_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first()
last_date_dt=dt.date.fromisoformat(last_date[0])
# last_date_dt=dt.datetime.strptime(last_date[0], 'YYYY-MM-DD')
prev_date_dt=last_date_dt-dt.timedelta(days=365)
prev_date_dt
# Perform a query to retrieve the data and precipitation scores
results = session.query(Measurement.date, Measurement.prcp).all()

# Save the query results as a Pandas DataFrame and set the index to the date column
pd.DataFrame(results)
df = pd.DataFrame()
#df.set_index('date')
df.head()
# Sort the dataframe by date


# Use Pandas Plotting with Matplotlib to plot the data




KeyError: "None of ['date'] are in the columns"

In [ ]:
# Use Pandas to calculate the summary statistics for the precipitation data


# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset


In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.


In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.


In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


# Close session

In [ ]:
# Close Session
session.close()